# **Semana 12: Modelos Avanzados de Clasificación**

## Ciencia de Datos en el Deporte - Análisis Avanzado

### 🚀 **Bloque 3: Análisis Avanzado y Modelado**

---

**Objetivos de Aprendizaje:**
- ✅ Dominar Random Forest para predicciones deportivas
- ✅ Implementar Support Vector Machines (SVM)
- ✅ Comparar múltiples modelos de clasificación
- ✅ Entender ensemble methods y votación
- ✅ Seleccionar el mejor modelo para cada situación

**Herramientas:**
- 🐍 Python
- 🤖 Scikit-learn (Random Forest, SVM, Ensemble)
- 📊 Pandas (preparación de datos)
- 📈 Matplotlib/Seaborn (comparación visual)
- 🔍 Cross-validation (validación cruzada)

---

## 1. Repaso: ¿Dónde Quedamos?

### 1.1 Lo que Aprendimos en la Semana 11

En la semana anterior implementamos nuestro **primer modelo predictivo**:

✅ **Regresión Logística** para predecir resultados de fútbol  
✅ **Preparación de datos** con variables relevantes  
✅ **División entrenamiento/prueba** para evaluación correcta  
✅ **Evaluación básica** con matriz de confusión  

### 1.2 ¿Por qué Necesitamos Modelos Más Avanzados?

La **Regresión Logística** es excelente para empezar, pero tiene limitaciones:

#### **Limitaciones de Regresión Logística:**
- ⚠️ **Asume relaciones lineales**: No captura patrones complejos
- ⚠️ **Sensible a outliers**: Valores extremos afectan mucho
- ⚠️ **Requiere preprocesamiento**: Variables deben estar escaladas

#### **Ventajas de Modelos Avanzados:**
- ✅ **Capturan patrones complejos**: Relaciones no lineales
- ✅ **Más robustos**: Manejan mejor datos ruidosos
- ✅ **Mayor precisión**: Especialmente con datasets grandes

### 1.3 Modelos que Exploraremos Hoy

1. **Random Forest** 🌲: Bosques de árboles de decisión
2. **Support Vector Machine (SVM)** ⚡: Separación óptima de clases
3. **Ensemble Voting** 🗳️: Combinación de múltiples modelos
4. **Comparación sistemática** 📊: ¿Cuál es mejor?

## 2. Preparación del Entorno

### 2.1 Importar Librerías y Cargar Datos

Empezaremos cargando las herramientas necesarias y recreando nuestros datos:

In [1]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Librerías de machine learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Configuración de gráficos
sns.set_theme(style="whitegrid", palette="Set2")
plt.rcParams['figure.figsize'] = (12, 8)

print("Librerías avanzadas de machine learning cargadas:")
print("Random Forest: Bosques de árboles de decisión")
print("SVM: Support Vector Machines")
print("Ensemble: Combinación de modelos")
print("Cross-validation: Validación cruzada")
print("\n¡Listos para modelos avanzados!")

Librerías avanzadas de machine learning cargadas:
Random Forest: Bosques de árboles de decisión
SVM: Support Vector Machines
Ensemble: Combinación de modelos
Cross-validation: Validación cruzada

¡Listos para modelos avanzados!


### 2.2 Recrear y Expandir el Dataset

Vamos a recrear nuestros datos de la semana pasada y añadir algunas variables nuevas:

In [2]:
# Recrear datos de partidos con más variables
np.random.seed(42)

# Aumentar el dataset para mejor entrenamiento
n_partidos = 1000

# Equipos con diferentes "niveles"
equipos_top = ['Real Madrid', 'Barcelona', 'Atletico']
equipos_mid = ['Valencia', 'Sevilla', 'Villarreal', 'Betis']
equipos_low = ['Athletic', 'Sociedad', 'Getafe', 'Levante', 'Osasuna']
todos_equipos = equipos_top + equipos_mid + equipos_low

# Función para asignar "calidad" al equipo
def calidad_equipo(equipo):
    if equipo in equipos_top:
        return 3  # Alta calidad
    elif equipo in equipos_mid:
        return 2  # Media calidad
    else:
        return 1  # Baja calidad

# Generar datos más realistas
datos_partidos = {
    'equipo_local': np.random.choice(todos_equipos, n_partidos),
    'equipo_visitante': np.random.choice(todos_equipos, n_partidos),
    'goles_local_ultimos_5': np.random.randint(0, 15, n_partidos),
    'goles_visitante_ultimos_5': np.random.randint(0, 15, n_partidos),
    'victorias_local_ultimos_5': np.random.randint(0, 6, n_partidos),
    'victorias_visitante_ultimos_5': np.random.randint(0, 6, n_partidos),
    'posicion_liga_local': np.random.randint(1, 21, n_partidos),
    'posicion_liga_visitante': np.random.randint(1, 21, n_partidos),
    'es_derbi': np.random.choice([0, 1], n_partidos, p=[0.85, 0.15]),
    'diferencia_valor_mercado': np.random.normal(0, 20, n_partidos),  # NUEVA: Diferencia en millones €
    'lesionados_local': np.random.randint(0, 5, n_partidos),  # NUEVA: Jugadores lesionados
    'lesionados_visitante': np.random.randint(0, 5, n_partidos),
    'dias_descanso_local': np.random.randint(2, 8, n_partidos),  # NUEVA: Días de descanso
    'dias_descanso_visitante': np.random.randint(2, 8, n_partidos)
}

# Crear DataFrame
df_partidos = pd.DataFrame(datos_partidos)

# Eliminar partidos donde un equipo juega contra sí mismo
df_partidos = df_partidos[df_partidos['equipo_local'] != df_partidos['equipo_visitante']]

# Añadir calidad de equipos
df_partidos['calidad_local'] = df_partidos['equipo_local'].apply(calidad_equipo)
df_partidos['calidad_visitante'] = df_partidos['equipo_visitante'].apply(calidad_equipo)

print(f"Dataset expandido creado con {len(df_partidos)} partidos")
print(f"Variables totales: {len(df_partidos.columns)}")
print("\nNuevas variables añadidas:")
print("- Diferencia en valor de mercado")
print("- Jugadores lesionados")
print("- Días de descanso")
print("- Calidad del equipo")

print("\nPrimeros 3 partidos:")
print(df_partidos.head(3))

Dataset expandido creado con 926 partidos
Variables totales: 16

Nuevas variables añadidas:
- Diferencia en valor de mercado
- Jugadores lesionados
- Días de descanso
- Calidad del equipo

Primeros 3 partidos:
  equipo_local equipo_visitante  goles_local_ultimos_5  \
0        Betis         Valencia                      1   
1     Valencia          Levante                      5   
2      Levante         Athletic                      1   

   goles_visitante_ultimos_5  victorias_local_ultimos_5  \
0                          6                          3   
1                          6                          3   
2                         11                          0   

   victorias_visitante_ultimos_5  posicion_liga_local  \
0                              0                    9   
1                              2                   15   
2                              0                   16   

   posicion_liga_visitante  es_derbi  diferencia_valor_mercado  \
0                       1

### 2.3 Crear Variable Objetivo Mejorada

Ahora usaremos una lógica más sofisticada que incorpore las nuevas variables:

In [3]:
# Función mejorada para simular resultados
def simular_resultado_avanzado(row):
    """
    Simula resultado con lógica más realista incluyendo nuevas variables
    """
    # Fortaleza básica (como antes)
    fortaleza_local = (
        row['goles_local_ultimos_5'] * 0.2 +
        row['victorias_local_ultimos_5'] * 0.3 +
        (21 - row['posicion_liga_local']) * 0.2 +
        row['calidad_local'] * 3 +  # NUEVO: Calidad del equipo
        3  # Ventaja de local
    )
    
    fortaleza_visitante = (
        row['goles_visitante_ultimos_5'] * 0.2 +
        row['victorias_visitante_ultimos_5'] * 0.3 +
        (21 - row['posicion_liga_visitante']) * 0.2 +
        row['calidad_visitante'] * 3
    )
    
    # Ajustes por nuevas variables
    fortaleza_local += row['diferencia_valor_mercado'] * 0.1  # Más valor = más fuerte
    fortaleza_local -= row['lesionados_local'] * 0.5  # Lesiones debilitan
    fortaleza_local += (row['dias_descanso_local'] - 4) * 0.3  # Descanso óptimo = 4 días
    
    fortaleza_visitante -= row['diferencia_valor_mercado'] * 0.1
    fortaleza_visitante -= row['lesionados_visitante'] * 0.5
    fortaleza_visitante += (row['dias_descanso_visitante'] - 4) * 0.3
    
    # Factor derbi (más impredecible)
    if row['es_derbi'] == 1:
        factor_aleatorio = np.random.normal(0, 3)  # Más variabilidad
    else:
        factor_aleatorio = np.random.normal(0, 2)
    
    diferencia = fortaleza_local - fortaleza_visitante + factor_aleatorio
    
    # Determinar resultado con umbrales ajustados
    if diferencia > 2.0:
        return 'Victoria_Local'
    elif diferencia < -2.0:
        return 'Victoria_Visitante'
    else:
        return 'Empate'

# Aplicar la función
df_partidos['resultado'] = df_partidos.apply(simular_resultado_avanzado, axis=1)

# Ver distribución
print("Distribución de resultados (mejorada):")
distribucion = df_partidos['resultado'].value_counts()
print(distribucion)
print(f"\nPorcentajes:")
for resultado, cantidad in distribucion.items():
    porcentaje = (cantidad / len(df_partidos)) * 100
    print(f"- {resultado}: {porcentaje:.1f}%")

# Verificar que tenemos datos balanceados
print(f"\nDataset final: {len(df_partidos)} partidos con distribución realista")

Distribución de resultados (mejorada):
resultado
Victoria_Local        510
Empate                215
Victoria_Visitante    201
Name: count, dtype: int64

Porcentajes:
- Victoria_Local: 55.1%
- Empate: 23.2%
- Victoria_Visitante: 21.7%

Dataset final: 926 partidos con distribución realista


## 3. Random Forest: Bosques de Árboles de Decisión

### 3.1 ¿Qué es Random Forest?

**Random Forest** es como tener un **comité de expertos** tomando decisiones:

#### **Analogía Futbolística:**
Imagina que tienes 100 analistas deportivos, cada uno con una opinión sobre quién ganará:
- 🧠 **Cada analista** ve solo parte de la información
- 🗳️ **Todos votan** su predicción
- 🏆 **La mayoría gana** - esa es la predicción final

#### **Ventajas de Random Forest:**
- ✅ **Muy preciso**: Combina múltiples modelos
- ✅ **Robusto**: No se "sobreajusta" fácilmente
- ✅ **Maneja datos faltantes**: Automáticamente
- ✅ **Indica importancia**: Qué variables son más relevantes

### 3.2 Implementación de Random Forest

In [4]:
# Preparar datos para Random Forest
features_rf = [
    'goles_local_ultimos_5', 'goles_visitante_ultimos_5',
    'victorias_local_ultimos_5', 'victorias_visitante_ultimos_5',
    'posicion_liga_local', 'posicion_liga_visitante',
    'calidad_local', 'calidad_visitante',
    'diferencia_valor_mercado', 'lesionados_local', 'lesionados_visitante',
    'dias_descanso_local', 'dias_descanso_visitante', 'es_derbi'
]

X = df_partidos[features_rf]
y = df_partidos['resultado']

# División entrenamiento/prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Crear y entrenar Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,  # 100 árboles en el bosque
    max_depth=10,      # Profundidad máxima de cada árbol
    random_state=42,
    min_samples_split=5,  # Mínimo de muestras para dividir
    min_samples_leaf=2    # Mínimo de muestras en cada hoja
)

print("Entrenando Random Forest (100 árboles)...")
rf_model.fit(X_train, y_train)
print("¡Random Forest entrenado!")

# Hacer predicciones
rf_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)

print(f"\nPrecisión de Random Forest: {rf_accuracy:.2%}")

# Mostrar importancia de variables
importancias = pd.DataFrame({
    'Variable': features_rf,
    'Importancia': rf_model.feature_importances_
}).sort_values('Importancia', ascending=False)

print("\nImportancia de variables en Random Forest:")
for i, row in importancias.head(5).iterrows():
    print(f"{row['Variable']}: {row['Importancia']:.3f}")

Entrenando Random Forest (100 árboles)...
¡Random Forest entrenado!

Precisión de Random Forest: 72.58%

Importancia de variables en Random Forest:
diferencia_valor_mercado: 0.301
calidad_visitante: 0.102
posicion_liga_visitante: 0.084
calidad_local: 0.080
posicion_liga_local: 0.075


## 4. Support Vector Machine (SVM)

### 4.1 ¿Qué es SVM?

**SVM** busca la **mejor línea** para separar las clases:

#### **Analogía Futbolística:**
Imagina que tienes que **separar equipos** en un campo:
- 🏟️ **Campo**: Espacio de características (goles, posición, etc.)
- 📏 **Línea**: Frontera que separa "equipos que ganan" de "equipos que pierden"
- 🎯 **Objetivo**: Encontrar la línea que mejor separe los grupos

#### **Ventajas de SVM:**
- ✅ **Muy preciso**: Especialmente con datos complejos
- ✅ **Maneja no linealidad**: Con "kernels" (trucos matemáticos)
- ✅ **Robusto con outliers**: No se deja influir por casos extremos

#### **Desventajas:**
- ⚠️ **Lento con datos grandes**: Puede tardar en entrenar
- ⚠️ **Difícil de interpretar**: "Caja negra"

### 4.2 Implementación de SVM

In [5]:
# SVM requiere datos escalados (normalizados)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear SVM con kernel RBF (Radial Basis Function)
svm_model = SVC(
    kernel='rbf',        # Kernel para capturar patrones no lineales
    C=1.0,              # Parámetro de regularización
    probability=True,    # Para obtener probabilidades
    random_state=42
)

print("Entrenando SVM (puede tardar un poco)...")
svm_model.fit(X_train_scaled, y_train)
print("¡SVM entrenado!")

# Hacer predicciones
svm_pred = svm_model.predict(X_test_scaled)
svm_accuracy = accuracy_score(y_test, svm_pred)

print(f"\nPrecisión de SVM: {svm_accuracy:.2%}")

# Información del modelo
print(f"Vectores de soporte utilizados: {svm_model.n_support_}")
print(f"Total de vectores de soporte: {sum(svm_model.n_support_)}")

Entrenando SVM (puede tardar un poco)...
¡SVM entrenado!

Precisión de SVM: 80.65%
Vectores de soporte utilizados: [172 183 124]
Total de vectores de soporte: 479


## 5. Comparación de Modelos

### 5.1 Comparar con Regresión Logística

Vamos a entrenar también regresión logística para comparar los tres modelos:

In [ ]:
# Entrenar Regresión Logística para comparación
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)  # Usa datos escalados
lr_pred = lr_model.predict(X_test_scaled)
lr_accuracy = accuracy_score(y_test, lr_pred)

# Crear comparación visual
modelos = ['Regresión Logística', 'Random Forest', 'SVM']
precisiones = [lr_accuracy, rf_accuracy, svm_accuracy]

# Gráfico de comparación
plt.figure(figsize=(10, 6))
bars = plt.bar(modelos, precisiones, color=['skyblue', 'lightgreen', 'lightcoral'])
plt.title('Comparación de Precisión entre Modelos', size=16, pad=20)
plt.ylabel('Precisión (%)')
plt.ylim(0, 1)

# Añadir valores encima de las barras
for bar, precision in zip(bars, precisiones):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{precision:.2%}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Resumen de resultados
print("COMPARACIÓN DE MODELOS:")
print("=" * 40)
for modelo, precision in zip(modelos, precisiones):
    print(f"{modelo:20}: {precision:.2%}")

# Encontrar el mejor modelo
mejor_idx = np.argmax(precisiones)
print(f"\n🏆 MEJOR MODELO: {modelos[mejor_idx]} ({precisiones[mejor_idx]:.2%})")

### 5.2 Análisis Detallado por Modelo

Veamos cómo se desempeña cada modelo en cada tipo de resultado:

In [ ]:
# Crear matriz de confusión para cada modelo
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

modelos_pred = [lr_pred, rf_pred, svm_pred]
nombres = ['Regresión Logística', 'Random Forest', 'SVM']

for i, (pred, nombre) in enumerate(zip(modelos_pred, nombres)):
    cm = confusion_matrix(y_test, pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                xticklabels=['Empate', 'Victoria_Local', 'Victoria_Visitante'],
                yticklabels=['Empate', 'Victoria_Local', 'Victoria_Visitante'])
    axes[i].set_title(f'{nombre}\nPrecisión: {precisiones[i]:.2%}')
    axes[i].set_ylabel('Realidad' if i == 0 else '')
    axes[i].set_xlabel('Predicción')

plt.tight_layout()
plt.show()

# Reportes detallados
print("REPORTES DETALLADOS POR CLASE:")
print("=" * 50)

for nombre, pred in zip(nombres, modelos_pred):
    print(f"\n{nombre.upper()}:")
    print(classification_report(y_test, pred, target_names=['Empate', 'Victoria_Local', 'Victoria_Visitante']))

## 6. Ensemble Methods: Combinando Modelos

### 6.1 ¿Qué son los Ensemble Methods?

Los **Ensemble Methods** combinan múltiples modelos para obtener mejores resultados:

#### **Analogía del Comité de Expertos:**
- 🧠 **Experto en estadísticas**: Regresión Logística
- 🌲 **Experto en patrones**: Random Forest  
- ⚡ **Experto en separación**: SVM
- 🗳️ **Votación final**: Combinan sus opiniones

#### **Tipos de Ensemble:**
- **Voting (Votación)**: Cada modelo vota, gana la mayoría
- **Weighted Voting**: Algunos modelos tienen más peso
- **Soft Voting**: Usa probabilidades en lugar de votos duros

### 6.2 Implementar Voting Classifier

In [ ]:
# Crear ensemble con los tres modelos
ensemble_model = VotingClassifier(
    estimators=[
        ('logistic', lr_model),
        ('random_forest', rf_model),
        ('svm', svm_model)
    ],
    voting='soft'  # Usa probabilidades para votación más sofisticada
)

print("Entrenando Ensemble Model (combinación de los 3 modelos)...")

# Nota: Random Forest usa datos originales, otros usan escalados
# Necesitamos entrenar de forma especial
from sklearn.pipeline import Pipeline

# Crear pipelines para mantener consistencia
lr_pipeline = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(random_state=42, max_iter=1000))])
svm_pipeline = Pipeline([('scaler', StandardScaler()), ('svm', SVC(kernel='rbf', C=1.0, probability=True, random_state=42))])

# Ensemble con pipelines
ensemble_model = VotingClassifier(
    estimators=[
        ('lr_pipeline', lr_pipeline),
        ('rf', rf_model),
        ('svm_pipeline', svm_pipeline)
    ],
    voting='soft'
)

# Entrenar ensemble
ensemble_model.fit(X_train, y_train)
print("¡Ensemble Model entrenado!")

# Hacer predicciones
ensemble_pred = ensemble_model.predict(X_test)
ensemble_accuracy = accuracy_score(y_test, ensemble_pred)

print(f"\nPrecisión del Ensemble: {ensemble_accuracy:.2%}")

# Comparación final
modelos_final = ['Regresión Logística', 'Random Forest', 'SVM', 'Ensemble']
precisiones_final = [lr_accuracy, rf_accuracy, svm_accuracy, ensemble_accuracy]

print("\nCOMPARACIÓN FINAL:")
print("=" * 30)
for modelo, precision in zip(modelos_final, precisiones_final):
    print(f"{modelo:20}: {precision:.2%}")

mejora = ensemble_accuracy - max(precisiones_final[:-1])
print(f"\nMejora del Ensemble: +{mejora:.2%}")

### 6.3 Visualización Final de Resultados

In [ ]:
# Gráfico final comparativo
plt.figure(figsize=(12, 8))

# Subplot 1: Comparación de precisiones
plt.subplot(2, 2, 1)
colors = ['skyblue', 'lightgreen', 'lightcoral', 'gold']
bars = plt.bar(modelos_final, precisiones_final, color=colors)
plt.title('Comparación Final de Modelos')
plt.ylabel('Precisión')
plt.xticks(rotation=45)
for bar, precision in zip(bars, precisiones_final):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, 
             f'{precision:.2%}', ha='center', va='bottom', fontweight='bold')

# Subplot 2: Matriz de confusión del mejor modelo
plt.subplot(2, 2, 2)
mejor_pred = ensemble_pred if ensemble_accuracy == max(precisiones_final) else modelos_pred[np.argmax(precisiones_final[:-1])]
cm_mejor = confusion_matrix(y_test, mejor_pred)
sns.heatmap(cm_mejor, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Empate', 'Victoria_Local', 'Victoria_Visitante'],
            yticklabels=['Empate', 'Victoria_Local', 'Victoria_Visitante'])
plt.title('Matriz de Confusión - Mejor Modelo')
plt.ylabel('Realidad')
plt.xlabel('Predicción')

# Subplot 3: Importancia de variables (Random Forest)
plt.subplot(2, 2, 3)
importancias_top = importancias.head(8)
plt.barh(importancias_top['Variable'], importancias_top['Importancia'], color='lightblue')
plt.title('Variables Más Importantes (Random Forest)')
plt.xlabel('Importancia')

# Subplot 4: Distribución de predicciones del ensemble
plt.subplot(2, 2, 4)
pred_counts = pd.Series(ensemble_pred).value_counts()
plt.pie(pred_counts.values, labels=pred_counts.index, autopct='%1.1f%%', colors=['lightcoral', 'lightgreen', 'lightyellow'])
plt.title('Distribución de Predicciones\n(Ensemble Model)')

plt.tight_layout()
plt.show()

# Resumen de insights
print("INSIGHTS PRINCIPALES:")
print("=" * 40)
print(f"1. Mejor modelo individual: {modelos[np.argmax(precisiones)]}")
print(f"2. Ensemble {'mejora' if ensemble_accuracy > max(precisiones) else 'no mejora'} la precisión")
print(f"3. Variables más importantes: {', '.join(importancias.head(3)['Variable'].tolist())}")
print(f"4. Precisión promedio de todos los modelos: {np.mean(precisiones_final):.2%}")

## 7. Validación Cruzada

### 7.1 ¿Qué es la Validación Cruzada?

La **validación cruzada** es como hacer **múltiples exámenes** para estar seguro del rendimiento:

#### **Analogía de Evaluación Deportiva:**
- 🏟️ **Un partido**: Como usar solo train/test split
- 🏆 **Temporada completa**: Como validación cruzada
- 📊 **Promedio de temporada**: Resultado más confiable

#### **Proceso:**
1. Dividir datos en 5 partes (folds)
2. Entrenar con 4 partes, probar con 1
3. Repetir 5 veces
4. Promediar resultados

### 7.2 Implementar Validación Cruzada

In [ ]:
# Validación cruzada para todos los modelos
print("Realizando validación cruzada (5-fold)...")
print("Esto puede tardar un momento...\n")

# Modelos para validar
modelos_cv = {
    'Logistic Regression': lr_pipeline,
    'Random Forest': rf_model,
    'SVM': svm_pipeline,
    'Ensemble': ensemble_model
}

resultados_cv = {}

for nombre, modelo in modelos_cv.items():
    # 5-fold cross validation
    scores = cross_val_score(modelo, X_train, y_train, cv=5, scoring='accuracy')
    resultados_cv[nombre] = {
        'scores': scores,
        'mean': scores.mean(),
        'std': scores.std()
    }
    print(f"{nombre:20}: {scores.mean():.3f} (±{scores.std():.3f})")

# Visualizar resultados de validación cruzada
plt.figure(figsize=(10, 6))
nombres_cv = list(resultados_cv.keys())
scores_cv = [resultados_cv[nombre]['scores'] for nombre in nombres_cv]

# Box plot de los scores
bp = plt.boxplot(scores_cv, labels=nombres_cv, patch_artist=True)
colors = ['skyblue', 'lightgreen', 'lightcoral', 'gold']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

plt.title('Validación Cruzada - Distribución de Scores')
plt.ylabel('Accuracy')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Análisis de estabilidad
print("\nANÁLISIS DE ESTABILIDAD:")
print("=" * 30)
for nombre, resultado in resultados_cv.items():
    estabilidad = "Alta" if resultado['std'] < 0.02 else "Media" if resultado['std'] < 0.04 else "Baja"
    print(f"{nombre:20}: {estabilidad} (std: {resultado['std']:.3f})")

# Mejor modelo según validación cruzada
mejor_cv = max(resultados_cv.items(), key=lambda x: x[1]['mean'])
print(f"\n🏆 MEJOR MODELO (CV): {mejor_cv[0]} ({mejor_cv[1]['mean']:.3f})")

## 8. Predicción Práctica

### 8.1 Usar el Mejor Modelo para Predicciones

Ahora usemos nuestro mejor modelo para hacer predicciones realistas:

In [ ]:
# Usar el mejor modelo según validación cruzada
mejor_modelo = modelos_cv[mejor_cv[0]]

# Crear scenarios de partidos realistas
scenarios = [
    {
        'nombre': 'Clásico: Real Madrid vs Barcelona',
        'goles_local_ultimos_5': 13, 'goles_visitante_ultimos_5': 12,
        'victorias_local_ultimos_5': 4, 'victorias_visitante_ultimos_5': 4,
        'posicion_liga_local': 1, 'posicion_liga_visitante': 2,
        'calidad_local': 3, 'calidad_visitante': 3,
        'diferencia_valor_mercado': 5, 'lesionados_local': 1, 'lesionados_visitante': 2,
        'dias_descanso_local': 4, 'dias_descanso_visitante': 3, 'es_derbi': 1
    },
    {
        'nombre': 'David vs Goliat: Getafe vs Real Madrid',
        'goles_local_ultimos_5': 4, 'goles_visitante_ultimos_5': 14,
        'victorias_local_ultimos_5': 1, 'victorias_visitante_ultimos_5': 5,
        'posicion_liga_local': 15, 'posicion_liga_visitante': 1,
        'calidad_local': 1, 'calidad_visitante': 3,
        'diferencia_valor_mercado': -50, 'lesionados_local': 3, 'lesionados_visitante': 0,
        'dias_descanso_local': 7, 'dias_descanso_visitante': 3, 'es_derbi': 0
    },
    {
        'nombre': 'Partido Parejo: Valencia vs Sevilla',
        'goles_local_ultimos_5': 8, 'goles_visitante_ultimos_5': 9,
        'victorias_local_ultimos_5': 3, 'victorias_visitante_ultimos_5': 3,
        'posicion_liga_local': 7, 'posicion_liga_visitante': 6,
        'calidad_local': 2, 'calidad_visitante': 2,
        'diferencia_valor_mercado': -2, 'lesionados_local': 2, 'lesionados_visitante': 1,
        'dias_descanso_local': 4, 'dias_descanso_visitante': 4, 'es_derbi': 0
    }
]

print("PREDICCIONES CON EL MEJOR MODELO:")
print("=" * 50)

for scenario in scenarios:
    nombre = scenario.pop('nombre')
    
    # Crear DataFrame para predicción
    partido_df = pd.DataFrame([scenario])
    
    # Hacer predicción
    prediccion = mejor_modelo.predict(partido_df)[0]
    probabilidades = mejor_modelo.predict_proba(partido_df)[0]
    
    print(f"\n{nombre.upper()}:")
    print(f"Predicción: {prediccion}")
    
    # Mostrar probabilidades
    clases = mejor_modelo.classes_ if hasattr(mejor_modelo, 'classes_') else ['Empate', 'Victoria_Local', 'Victoria_Visitante']
    print("Probabilidades:")
    for clase, prob in zip(clases, probabilidades):
        print(f"  - {clase}: {prob:.1%}")
    
    # Nivel de confianza
    max_prob = max(probabilidades)
    if max_prob > 0.7:
        confianza = "Muy Alta"
    elif max_prob > 0.5:
        confianza = "Alta"
    elif max_prob > 0.4:
        confianza = "Media"
    else:
        confianza = "Baja"
    
    print(f"  Confianza: {confianza} ({max_prob:.1%})")

print("\n" + "="*50)
print(f"Modelo utilizado: {mejor_cv[0]}")
print(f"Precisión en validación cruzada: {mejor_cv[1]['mean']:.2%}")

## 9. Resumen y Conclusiones

### 9.1 Lo que Aprendimos Hoy

✅ **Modelos Avanzados de Clasificación**:
- **Random Forest**: Bosques de árboles para mayor precisión
- **SVM**: Separación óptima de clases con kernels
- **Ensemble Methods**: Combinación de modelos para mejores resultados

✅ **Técnicas de Evaluación**:
- **Comparación sistemática** de múltiples modelos
- **Validación cruzada** para resultados más confiables
- **Análisis de estabilidad** y robustez

✅ **Aplicación Práctica**:
- **Predicciones realistas** en scenarios deportivos
- **Interpretación de probabilidades** y confianza
- **Selección del mejor modelo** según criterios objetivos

### 9.2 Conceptos Clave para Recordar

🌲 **Random Forest**: "Sabiduría de las multitudes" - muchos árboles votan

⚡ **SVM**: Encuentra la mejor frontera para separar clases

🗳️ **Ensemble**: Combinar expertos para decisiones más acertadas

🔄 **Validación Cruzada**: Múltiples evaluaciones para mayor confianza

📊 **Feature Importance**: Qué variables realmente importan

### 9.3 Comparación Final de Modelos

| Modelo | Ventajas | Desventajas | Cuándo Usar |
|--------|----------|-------------|-------------|
| **Regresión Logística** | Simple, rápido, interpretable | Solo relaciones lineales | Baseline, datos pequeños |
| **Random Forest** | Robusto, maneja no linealidad | Menos interpretable | Datos medianos, muchas variables |
| **SVM** | Muy preciso, maneja complejidad | Lento, "caja negra" | Datos complejos, alta precisión |
| **Ensemble** | Combina fortalezas | Más complejo, más lento | Máxima precisión posible |

### 9.4 Recomendaciones Prácticas

#### **Para Proyectos Reales:**
1. **Empieza simple**: Regresión Logística como baseline
2. **Prueba Random Forest**: Buen equilibrio precisión/interpretabilidad
3. **Considera SVM**: Si necesitas máxima precisión
4. **Usa Ensemble**: Para competencias o aplicaciones críticas

#### **Para Optimizar Resultados:**
- 🔍 **Feature Engineering**: Crear variables más informativas
- ⚙️ **Hyperparameter Tuning**: Ajustar parámetros de modelos
- 📊 **Más datos**: Siempre ayuda (especialmente para SVM y RF)
- 🧹 **Limpieza de datos**: Calidad > Cantidad

### 9.5 ¿Qué Viene Después?

En las siguientes semanas profundizaremos en:
- **Semana 13**: Métricas avanzadas (ROC-AUC, Precision-Recall)
- **Semana 14**: Feature Engineering y optimización de hiperparámetros
- **Semana 15**: Proyecto final integrador

### 9.6 Ejercicio para Practicar

**Desafío**: 
1. Modifica los scenarios de predicción y observa cómo cambian los resultados
2. ¿Qué variables tienen más impacto en las predicciones?
3. ¿En qué scenarios el ensemble funciona mejor que los modelos individuales?

¡Excelente trabajo! Has dominado los modelos avanzados de clasificación para análisis deportivo. 🏆⚽🤖